# N Smallest Capitalization Stock

1. Set the number of stocks want to trade, N.
1. 设定好要交易的股票数量stocksnum

2. Find the first N number of stocks with the smallest market capitalization.
2. 每天找出市值排名最小的前stocksnum只股票作为要买入的股票。

3. Sell the stock if the price of the current stock holding is not small enough anymore (no in the list of N stocks)
3. 若已持有的股票的市值已经不够小而不在要买入的股票中，则卖出这些股票。

4. Buy the new stocks from the current list of N stocks with the smallest market capitalization.
4. 买入要买入的股票，买入金额为当前可用资金的stocksnum分之一。

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Read in the sample data csv  (use df scraped from yahoo finance)
# Use Close and Volume value to get daily market captalization for each stock 
# Market Cap = Close * Volume for each stock
df = pd.read_csv('SampleData.csv')
columns = df.columns.tolist()

for x in columns[1:26]:
    index = columns.index(x)
    if index < 26:
        market = df[df.columns[index]] * df[df.columns[index+25]]
        df[str(x) + '_' + 'Mkt'] = market


# Make the 'Date' row index
df = df.set_index('Date')

In [3]:
# Only capture the Market Cap as needed
# [:, 50:76] Depends on the dataset (50:76) numbers need to change
df_small = df.iloc[:, 50:76]
small = df_small.T

In [96]:
# small1 = small.sort_values([small.columns[0]])
# list1 = np.array(small1.index[0:5]).tolist()
# choice = np.array(small1.index[0:stocknum]).tolist()
# choice1 = [i.split('_')[0] for i in choice]
# small2 = small.sort_values([small.columns[1]])
# list2 = np.array(small2.index[0:5]).tolist()
# print(small1)
#print(choice)
#print(small2)
# a = []
# a.append(list1)
# a.append(list2)
# print(a)

FINAL USE!!!!

In [136]:
def second_day(x): # x - the first day!!!!!!
    position = small.columns.tolist().index(x) # get the column index of second day of the date x input
    y = small.columns[position + 1]
    small2 = small.sort_values([y]) # sort according to date x
    choice2 = np.array(small2.index[0:stocknum]).tolist() # get the smallest N(5) stocks of the second day
    choice2 = [i.split('_')[0] for i in choice2]
    ones = np.ones(stocknum, dtype=int) # give all 1 to  to buy for each stock
    a = dict(zip(choice2, ones))

    
    ### function to get the initial basket from date x ###
    def initial_day(x, cashpool, stocknum):
        position = small.columns.tolist().index(x) # get the column index of the date x input
        small1 = small.sort_values([x]) # sort according to date x
        choice = np.array(small1.index[0:stocknum]).tolist() # get the smallest N(5) stocks of the day
        choice = [i.split('_')[0] for i in choice]

        # get the column name I want to find the close price for each stock on day x
        price = [i.split('_')[0] + "_" + 'Close' for i in choice]

        # find the close price for each stock on day x
        close = []
        for i in range(len(price)):
            b = df.loc[x, price[i]]
            close.append(b)
            invest = cashpool / stocknum
            share = [int(invest / i) for i in close]
            buy_dict = dict(zip(choice, share)) # update buy_list for INITIAL day x
        return buy_dict

    
    ### FINALIZED 3 dicts to return at the end ###
    buy = {k:a[k] for k in a if k not in buy_dict} # in second day's purchase list, not in day x, buy
    keep = {k:0 for k in a if k in buy_dict} # in both days' purchase list, keep no change
    sell = {k:-buy_dict[k] for k,v in buy_dict.items() if k not in b} # in day x not in second day's purchase, sell 
    
    return (buy, keep, sell)
    

#     # get the column name I want to find the close price for each stock on day x
#     price2 = [i.split('_')[0] + '_' + 'Close' for i in choice2]
    
#     # find the close price for each stock on day x
#     close2 = []
#     for i in range(len(price2)):
#         b = df.loc[x, price2[i]]
#         close2.append(b)
#         invest = cashpool / stocknum
#         share2 = [int(invest / i) for i in close2]
#         buy_dict = dict(zip(choice2, share2)) # update buy_list for INITIAL day x
     
#     return buy_dict

In [135]:
second_day('2018-11-13')

({'X': 1, 'SIRI': 1}, {'LYG': 0, 'ABEV': 0, 'CHK': 0}, {'SWN': -3, 'FCX': -1})

TRIAL USE

In [76]:
def initial_day(x, cashpool, stocknum):
    position = small.columns.tolist().index(x) # get the column index of the date x input
    small1 = small.sort_values([x]) # sort according to date x
    choice = np.array(small1.index[0:stocknum]).tolist() # get the smallest N(5) stocks of the day
    choice = [i.split('_')[0] for i in choice]
    
    # get the column name I want to find the close price for each stock on day x
    price = [i.split('_')[0] + "_" + 'Close' for i in choice]
    
    # find the close price for each stock on day x
    close = []
    for i in range(len(price)):
        b = df.loc[x, price[i]]
        close.append(b)
        invest = cashpool / stocknum
        share = [int(invest / i) for i in close]
        buy_dict = dict(zip(choice, share)) # update buy_list for INITIAL day x
    return buy_dict

In [101]:
buy_dict = {'LYG': 6, 'ABEV': 4, 'SWN': 3, 'CHK': 5, 'FCX': 1}
a = {'LYG': 1, 'ABEV': 1, 'X': 1, 'CHK': 1, 'FCX': 1}

In [132]:
buy = {k:a[k] for k in a if k not in buy_dict} # in second day's purchase list, not in day x, buy
keep = {k:0 for k in a if k in buy_dict} # in both days' purchase list, keep no change
sell = {k:-buy_dict[k] for k,v in buy_dict.items() if k not in b} # in day x not in second day's purchase, sell 
# sellname = [i.split('_')[0] + '_' + 'Close' for i in key]
# sellprice = [df.loc['2018-11-13', sellname[i]] for i in range(len(sellname))]

{'LYG': 0, 'ABEV': 0, 'CHK': 0, 'FCX': 0}

In [72]:
initial_day('2018-11-12',100, 5)

{'LYG': 6, 'ABEV': 4, 'SWN': 3, 'CHK': 5, 'FCX': 1}

In [73]:
initial_day('2018-11-13',100, 5)

{'LYG': 6, 'ABEV': 4, 'X': 0, 'CHK': 5, 'FCX': 1}

In [57]:
# create portfolio_list for calculating returns
def portfolio_of_date(x):
    position = small.columns.tolist().index(x)
    
    buy = []
    for i in range (len(small.columns)):
        small1 = small.sort_values([a.columns[i]])
        ones = np.ones(5, dtype=int)
        choice = np.array(small1.index[0:5]).tolist()
        c = dict(zip(choice, ones))                      
        buy.append(c)
    
    # 'portfolio' is same as 'buy' on the first day(buy top 5 stocks whose market values are smallest)
    portfolio_list = [buy[0]]
    for i in range(len(buy) - 1):
        dictMerged = dict(buy[i], **buy[i+1]) # merge dictionarys
        for j in buy[i]:
            if j in buy[i+1]: # if stock A exists in the both portfolios, we set it to be 0 in the merged portfolio
                dictMerged[j] = 0 # 0 means no actions
                
            else:            # if stock A exists in the first portfolio but not in the second portfolio, we set it to be -1 in the merged portfolio
                dictMerged[j] = -1 # -1 means shorting the stock
                
                             # if stock A exists in the second portfolio but not in the first portfolio, we set it to be 1 in the merged portfolio
                             # 1 means longing the stock(same as 'buy')
                             
        portfolio_list.append(dictMerged)
    return portfolio_list[position] # getting portfolio_list during the trading days

In [64]:
a = portfolio_of_date('2016-11-28')